In [1]:
from keras.preprocessing.image import ImageDataGenerator

num_classes = 81
size = 32
batch_size = 16

train_data_dir = "./data/train"
test_data_dir = "./data/validation"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_data_dir, 
    target_size=(size,size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True    
)

test_data = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(size,size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 41322 images belonging to 81 classes.
Found 13877 images belonging to 81 classes.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten

model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(size,size,3), padding="same", activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(num_classes, activation="softmax"))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)         

In [4]:
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("./fruits_checkpoint_v1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1
)

earlystop = EarlyStopping(monitor="val_loss",
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
)

callbacks = [earlystop, checkpoint]

model.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=0.001), metrics=['accuracy'])

nb_train_samples = 41322
nb_test_samples = 13877
epochs = 5

history = model.fit_generator(train_data, 
                              steps_per_epoch=nb_train_samples//batch_size,
                              epochs=epochs,
                              verbose=1,
                              callbacks=callbacks,
                              validation_data=test_data,
                              validation_steps=nb_test_samples//batch_size)

Epoch 1/5
2582/2582 [==============================] - 137s 53ms/step - loss: 1.8453 - accuracy: 0.4399 - val_loss: 0.0167 - val_accuracy: 0.8451

Epoch 00001: val_loss improved from inf to 0.01666, saving model to ./fruits_checkpoint_v1.h5
Epoch 2/5
2582/2582 [==============================] - 159s 61ms/step - loss: 0.7580 - accuracy: 0.7565 - val_loss: 0.0063 - val_accuracy: 0.7718

Epoch 00002: val_loss improved from 0.01666 to 0.00632, saving model to ./fruits_checkpoint_v1.h5
Epoch 3/5
2582/2582 [==============================] - 165s 64ms/step - loss: 0.6089 - accuracy: 0.8163 - val_loss: 6.5118e-06 - val_accuracy: 0.8963

Epoch 00003: val_loss improved from 0.00632 to 0.00001, saving model to ./fruits_checkpoint_v1.h5
Epoch 4/5
2582/2582 [==============================] - 164s 63ms/step - loss: 0.6027 - accuracy: 0.8304 - val_loss: 0.0000e+00 - val_accuracy: 0.9202

Epoch 00004: val_loss improved from 0.00001 to 0.00000, saving model to ./fruits_checkpoint_v1.h5
Epoch 5/5
2582/2

In [5]:
score = model.evaluate_generator(test_data)
print("test loss ", score[0])
print("test acc", score[1])

test loss  1.0251987987430766e-06
test acc 0.9174172878265381
